# ADM-HW4: GROUP #10

#### Xavier Del Giudice


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  Valeria Avino| 1905974 |   avino.1905974@studenti.uniroma1.it                                      |
|  |  |                       |

## Import and Install required packages

In [2]:
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import re
import time

# 1. Recommendation System with LSH

## 1.1 Data Preparation

Reading the dataset and checking (if any) missing data.

In [11]:
#reading from our dataset
genome_scores = pd.read_csv('MovieLens/genome_scores.csv')
genome_tags = pd.read_csv('MovieLens/genome_tags.csv')
link = pd.read_csv('MovieLens/link.csv')
movie = pd.read_csv('MovieLens/movie.csv')
rating = pd.read_csv('MovieLens/rating.csv')
tags = pd.read_csv('MovieLens/tag.csv')

In [12]:
genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


In [13]:
#checking missing values
missing_values = genome_scores.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [14]:
genome_scores.duplicated().sum()

0

scores dataset has no missing values or duplicated rows

In [15]:
genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [16]:
missing_values = genome_tags.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [17]:
genome_tags.duplicated().sum()

0

In [18]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [19]:
missing_values = link.isnull().sum()
missing_values[missing_values > 0]

tmdbId    252
dtype: int64

In [20]:
link.duplicated().sum()

0

In [21]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [22]:
movie.duplicated().sum()

0

In [23]:
missing_values = movie.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [24]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [25]:
missing_values = rating.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [26]:
rating.duplicated().sum()

0

In [27]:
rating = rating.rename(columns={'timestamp':'rating_timestamp'})
rating.head()

,userId,movieId,rating,rating_timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [28]:
highest_rated = rating.loc[rating['rating'].idxmax(), 'movieId']
lowest_rated = rating.loc[rating['rating'].idxmin(), 'movieId']
#reading titles from movie dataset
print(f'The highest rated movie is {movie.loc[movie['movieId'] == highest_rated, 'title' ].values[0]} ')
print(f'The lowest rated movie is {movie.loc[movie['movieId'] == lowest_rated, 'title'].values[0]} ')

The highest rated movie is Lord of the Rings: The Fellowship of the Ring, The (2001) 
The lowest rated movie is Nemesis 2: Nebula (1995) 


In [29]:
# splitting genres column to do store it as a list
movie['genres'] = movie['genres'].str.split('|')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [30]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.2+ MB


In [31]:
missing_values = tags.isnull().sum()
missing_values[missing_values > 0]
#we could drop this rows or just keep them to check which user did not give a tag for which movies

tag    16
dtype: int64

In [32]:
tags.duplicated().sum()

0

In [33]:
tags = tags.rename(columns={'timestamp':'tag_timestamp'})
tags.head()

,userId,movieId,tag,tag_timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


### Merging the datasets on common columns

In [34]:
#merging tags and genom_tags on 'tag' column
merged_tags = pd.merge(tags, genome_tags, on ='tag', how = 'left')
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0
2,65,353,dark hero,2013-05-10 01:41:19,288.0
3,65,521,noir thriller,2013-05-10 01:39:43,712.0
4,65,592,dark hero,2013-05-10 01:41:18,288.0


In [35]:
#tag ids are converted into floats because od Nan values
#let's convert them back to nullable ints
merged_tags['tagId'] = merged_tags['tagId'].astype('int64',errors='ignore')

In [36]:
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0
2,65,353,dark hero,2013-05-10 01:41:19,288.0
3,65,521,noir thriller,2013-05-10 01:39:43,712.0
4,65,592,dark hero,2013-05-10 01:41:18,288.0


In [37]:
# joining the genome_scores to add the tag relevance
merged_tag = pd.merge(merged_tags, genome_scores, on = ['movieId', 'tagId'], how='left')

In [38]:
merged_tag.head()

,userId,movieId,tag,tag_timestamp,tagId,relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0,0.94950
2,65,353,dark hero,2013-05-10 01:41:19,288.0,0.99800
3,65,521,noir thriller,2013-05-10 01:39:43,712.0,0.98900
4,65,592,dark hero,2013-05-10 01:41:18,288.0,0.99775


In [39]:
df = pd.merge(rating, merged_tag, on = ['userId', 'movieId'], how = 'left')
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN


In [40]:
df = pd.merge( df, movie, on ='movieId', how='left')

In [41]:
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance,title,genres
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


In [42]:
df['userId'] = df['userId'].astype('int64')
df['tagId'] = df['tagId'].astype('int64', errors= 'ignore')
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance,title,genres
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


In [43]:
print(df.rating.isnull().sum())

0


In [3]:
df = pd.read_csv('complete_dataset.csv')

In [3]:
df.shape

(20265625, 10)

## 1.2 Minhash Signatures
Using the __`userId`__ and __`movieId`__ columns, implement your own **MinHash** function.
This function will hash each user's watched movie list, creating a representation that allows for quick comparisons of user similarities.

In order to create the signature matrix for our lists of seen movies we have implemented the following functions in the `functions.py` file:

+ The function `char_vec` creates a **characteristic vector** for a user's list of seen movies. It iterates over all possible movie IDs (`all_movies`) and assigns a `1` if a movie is in the user's seen list (`seen_movies`), and `0` otherwise. The result is a binary vector indicating the presence or absence of each movie for that user. 

+ The `minhash` function generates a **MinHash signature** for a given characteristic vector. This signature is a compact representation used in Locality Sensitive Hashing (LSH) to approximate Jaccard similarity efficiently. The function applies a set of **hash functions** to the indices of the vector, updating the signature values **only** for indices where the vector value is `1`. The result is a vector where each entry corresponds to the *minimum hashed index* encountered for each hash function.


In [4]:
from functions import char_vec, create_signature

In [11]:
# list of all movie ids
all_movies = df["movieId"].unique()
# selecting from df userId column and aggregating movieIds on it to create a list of user's seen movies 
user_seen_movies = df.groupby("userId")["movieId"].agg(set).reset_index().rename(columns={"movieId": "seen_movies"})
print(f"We  will work with {user_seen_movies.shape[0]} classes, since that is the number of users")

We  will work with 138493 classes, since that is the number of users


In [6]:
user_seen_movies.head()

,userId,seen_movies
0,1,"[2, 3081, 4105, 1036, 5146, 541, 29, 32, 4128,..."
1,2,"[3, 260, 2948, 2951, 1544, 1673, 266, 908, 245..."
2,3,"[512, 1, 3072, 2053, 2054, 2567, 1544, 2571, 1..."
3,4,"[6, 519, 520, 10, 19, 531, 32, 420, 165, 548, ..."
4,5,"[2, 515, 260, 1028, 648, 11, 140, 141, 780, 10..."


In [13]:
#sorting movie ids in ascending order
all_movies = sorted(all_movies)
print(f"Movie ids range from {all_movies[0]} to {all_movies[-1]}: we will work with {all_movies[-1]-all_movies[0]+1} shingles, each of which is one distinct movieId.")

Movie ids range from 1 to 131262: we will work with 131262 shingles, each of which is one distinct movieId.


* Compute the **characteristic vector** for each class, applying the `char_vec` function on `seen_movies` column.

In [14]:
user_seen_movies['char_vec'] = user_seen_movies['seen_movies'].apply(lambda x: char_vec(x, all_movies))

In [15]:
print(user_seen_movies.iloc[0]['char_vec'].toarray())

[[0 1 0 ... 0 0 0]]


In [ ]:
#just a check to remove
non_zero_indices = user_seen_movies.iloc[0]['char_vec'].indices
print(non_zero_indices+1)
print(sorted(user_seen_movies.iloc[0]['seen_movies']))

[   2   29   32   47   50  111  150  221  251  258  291  294  316  334
  364  538  584  588  646  903  908  991 1018 1058 1059 1068 1069 1076
 1114 1170 1172 1174 1176 1177 1183 1189 1190 1192 1194 1197 1213 1216
 1219 1222 1231 1232 1234 1235 1239 1251 1264 1276 1293 1305 1319 1321
 1329 1340 1344 1357 1479 1533 1687 1766 1837 1884 1911 1914 1938 2017
 2035 2055 2057 2060 2090 2091 2110 2111 2169 2204 2207 2458 2544 2560
 2563 2579 2598 2607 2631 2676 2677 2719 2787 2833 2859 2862 2874 2883
 2915 2944 2951 2995 3067 3179 3350 3388 3391 3400 3410 3797 3840 3903
 3904 3918 3934 4012 4035 4040 4133 4212 4352 4373 4477 4626 4659 4783
 4801 4816 4820 4846 4885 4898 4931 4944 4945 5051 5076 5444 5581 5699
 5718 5800 5854 5901 5995 6144 6235 6393 6430 6645 6646 6665 6698 6725
 6779 6890 6934 6935 7042 7053 7136 7276 7278 7313 7324 7329 7345 7456
 7770 7840 7861 7954 8008 8279 9754]
[2, 29, 32, 47, 50, 112, 151, 223, 253, 260, 293, 296, 318, 337, 367, 541, 589, 593, 653, 919, 924, 1009, 1036,

* Define multiple independent **hash functions** to permute shingles:
    In function `minhash` from `functions.py` we implement them as
    $$ h(x) = a_ix+b_i \text{ mod p } $$
    where $a$ and $b$ are random integers such that $a>0$ and $p$ is a prime larger then our number of shingles.


In [ ]:
import random

num_shingles = len(all_movies)
num_hash_functions = 3 #to be changed to compare results

# A large prime number greater than num_shingles
prime = 131273

# Generate random coefficients
random.seed(42)  # For reproducibility
a = np.random.randint(1, prime, size=num_hash_functions, dtype=np.int64)
b = np.random.randint(0, prime, size=num_hash_functions, dtype=np.int64)


* Now apply to each characteristic vector the `minhash` function to generate the associated **MinHash signature vector**.

In [ ]:
from function import minhash
user_seen_movies['minhash'] = user_seen_movies['char_vec'].apply(
    lambda x: minhash(x, num_hash_functions, a, b, prime)
)

In [ ]:
'''
# Initialize Spark session
spark = SparkSession.builder \
    .appName("MovieUserMatrix") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.python.worker.timeout", "12000") \
    .getOrCreate()

df_spark = spark.createDataFrame(user_seen_movies)

from functions import char_vec
all_movies_broadcast = spark.sparkContext.broadcast(all_movies)
char_vec_spark_udf = F.udf(char_vec, ArrayType(IntegerType()))
df_spark = df_spark.withColumn("char_vec", char_vec_spark_udf("seen_movies"))
'''

In [11]:
df_spark.printSchema()

root
 |-- userId: long (nullable = true)
 |-- seen_movies: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- char_vec: array (nullable = true)
 |    |-- element: integer (containsNull = true)



## 1.3 Locality-Sensitive Hashing (LSH)

### 1. Bucket Creation

### 2. Query

### 3. Movie Recommendation Logic

### 4. Final Recommendation

# 2. Grouping Movies Together!

## 2.1 Feature Engineering

## 2.2 Choose your features (variables)!

## 2.3 Clustering

## 2.4 Best Algorithm

# 3. Bonus Question

# 4. Algorithmic Question